# Prompt Template 提示词模板

## prompt template  字符模板

In [4]:
# 字符模板
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{area}专家")
prompt.format(area="AI模型")

'你是一个AI模型专家'

# 对话模板 ChatPromptTemplate

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
   ("system", "You are a helpful assistant."),
   ("ai", "I am an AI model."),
   ("human", "{user_input}"),
   ("assistant", "ai_response"),
  #  ("xxxx", "JJJJJJ")
  # Unexpected message type: xxxx.
  # Use one of 'human', 'user', 'ai', 'assistant', or 'system'.
])
chat_template.format_messages(user_input="Hello my AI!")

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am an AI model.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello my AI!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai_response', additional_kwargs={}, response_metadata={})]

In [21]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

sm = SystemMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'system'})
h_m = HumanMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'human'})
ai_m = AIMessage(content='I am an AI model.')
[sm, h_m, ai_m]


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'system'}, response_metadata={}),
 HumanMessage(content='You are a helpful assistant.', additional_kwargs={'name': 'human'}, response_metadata={}),
 AIMessage(content='I am an AI model.', additional_kwargs={}, response_metadata={})]

## ChatMassagePromptTemplate

In [30]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

pt = "愿{obj}荣光永存！"

sys_template = SystemMessagePromptTemplate.from_template(template=pt)
sys_template.format(obj="AAA")

hum_template = HumanMessagePromptTemplate.from_template(template=pt)
hum_template.format(obj="BBB")

ai_template = AIMessagePromptTemplate.from_template(template=pt)
ai_template.format(obj="CCCC")

fat_template = ChatMessagePromptTemplate.from_template(role='father', template=pt)
fat_template.format(obj="DDDD")

# [sys_template, hum_template, ai_template, fat_template]

ChatMessage(content='愿DDDD荣光永存！', additional_kwargs={}, response_metadata={}, role='father')

## 自定义模板 StringPromptTemplate

In [49]:
# AI函数解析器
from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数
def add(x: int, y: int) -> int:
    return x + y

# 创建一个自定义提示词模板
PROMPT="""\
你是一个函数解析器, 可以解析AI函数的参数和返回值。
根据给定的函数内容:{function_body},解析出一下内容：
参数列表, 返回值类型, 函数功能解释, 示例代码。
按照以下格式输出:

函数解析如下：
- 1. 函数名：{function_name}
- 2. 参数列表：
- 3. 返回值类型：
- 4. 函数体：
     {function_body}
- 5. 功能解释：
- 6. 示例代码：
"""

import inspect

def get_function_source(func):
    # 获取函数信息
    return inspect.getsource(func)


# 创建一个自定义的提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs):
        # 获得源代码信息
        func = kwargs.pop("function")
        func_info = get_function_source(func)
        return PROMPT.format(
            function_name=func.__name__,
            function_body=func_info,
        )


# 创建一个StringPromptTemplate实例
custom_pt = CustomPromptTemplate(input_variables=["function"])
cpt_str = custom_pt.format(function=add)

# 将生成的模板输入给AI
from langchain_ollama import ChatOllama

# 指定本地模型名称或路径
ollama_llm = ChatOllama(
    model="deepseek-r1:1.5b",  # 例如DeepSeek-R1 1.5B模型
    base_url="http://localhost:11434",  # Ollama默认地址
    temperature=0.5,
)
print(ollama_llm.predict(cpt_str))


<think>
嗯，我现在要解析一个Python函数。这个函数的定义是这样的：def add(x: int, y: int) -> int: return x + y。首先，我需要理解每个部分的意思。

函数名是add，这很简单，直接写上“add”。接下来是参数列表，里面有x和y两个参数，都是int类型。所以参数列表应该包括这两个变量，并且说明它们的类型。

然后是返回值类型，函数体最后面有个-> int，表示这个函数返回的是一个整数。这部分需要明确，不能出错。

接下来是函数的定义部分，也就是def add(x: int, y: int) -> int:. 这里x和y都是int类型的参数，并且整个函数返回的是int类型的结果。然后是函数体，直接写return x + y，这是简单的加法运算。

现在，我需要把这些信息按照用户的要求整理出来，按照指定的格式输出。首先列出每个部分的内容，确保不遗漏任何一个细节。

最后，检查一下是否所有部分都覆盖到了，有没有可能的地方被漏掉或者描述不清楚的地方。比如参数类型、返回值类型、函数名这些都很明确，没有问题。

总结一下，我需要按照用户给定的示例格式来填写，确保每个部分的信息准确无误地呈现出来。
</think>

函数解析如下：

- 1. 函数名：add
- 2. 参数列表：
- 3. 返回值类型：int
- 4. 函数体：
    def add(x: int, y: int) -> int:
        return x + y
- 5. 功能解释：这个函数接受两个整数参数x和y，并返回它们的和。
- 6. 示例代码：
    a = 3
    b = 4
    result = add(a, b)
    print(result)  # 输出是7
